importing relevant libraries. not sure how many of these will be useful later but better safe than sorry!

In [1]:
import os
import pandas as pd
import datetime as dt
import numpy as np
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler


# set configurations
pd.set_option('display.max_columns', 100)
sns.set_style("white")

# keras imports
import tensorflow as tf
import keras
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten
from keras.layers import GlobalMaxPooling1D, SpatialDropout1D, LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing.text import Tokenizer

# model imports
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score
import pickle
import joblib

Importing testing and training datasets and concating them into one file

In [ ]:
trainDataset = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

trainDataset['Istrain'] = 1
test['Istrain'] = 0
completeDataset = pd.concat([trainDataset, test], sort=True)

Took data from in-code dataset and imported into local variables/arrays/etc

In [ ]:
validation_splits = pd.DataFrame([
    ['Atlanta', 33.791, 33.835],
    ['Boston', 42.361, 42.383],
    ['Chicago', 41.921, 41.974],
    ['Philadelphia', 39.999, 40.046],
], columns=['City', 'l1', 'l2'])

directionCodes = {
    'N': 0,
    'NE': 1 / 4,
    'E': 1 / 2,
    'SE': 3 / 4,
    'S': 1,
    'SW': 5 / 4,
    'W': 3 / 2,
    'NW': 7 / 4
}

road_encoding = {
    'Road': 1,
    'Street': 2,
    'Avenue': 2,
    'Drive': 3,
    'Broad': 3,
    'Boulevard': 4
}

monthly_rainfall = {
    'Atlanta1': 5.02, 'Atlanta5': 3.95, 'Atlanta6': 3.63, 'Atlanta7': 5.12,
    'Atlanta8': 3.67, 'Atlanta9': 4.09, 'Atlanta10': 3.11, 'Atlanta11': 4.10,
    'Atlanta12': 3.82, 'Boston1': 3.92, 'Boston5': 3.24, 'Boston6': 3.22,
    'Boston7': 3.06, 'Boston8': 3.37, 'Boston9': 3.47, 'Boston10': 3.79,
    'Boston11': 3.98, 'Boston12': 3.73, 'Chicago1': 1.75, 'Chicago5': 3.38,
    'Chicago6': 3.63, 'Chicago7': 3.51, 'Chicago8': 4.62, 'Chicago9': 3.27,
    'Chicago10': 2.71, 'Chicago11': 3.01, 'Chicago12': 2.43,
    'Philadelphia1': 3.52, 'Philadelphia5': 3.88, 'Philadelphia6': 3.29,
    'Philadelphia7': 4.39, 'Philadelphia8': 3.82, 'Philadelphia9': 3.88,
    'Philadelphia10': 2.75, 'Philadelphia11': 3.16, 'Philadelphia12': 3.31
}

monthly_temperature = {
    'Atlanta1': 43, 'Atlanta5': 69, 'Atlanta6': 76, 'Atlanta7': 79,
    'Atlanta8': 78, 'Atlanta9': 73, 'Atlanta10': 62, 'Atlanta11': 53,
    'Atlanta12': 45, 'Boston1': 30, 'Boston5': 59, 'Boston6': 68, 'Boston7': 74,
    'Boston8': 73, 'Boston9': 66, 'Boston10': 55, 'Boston11': 45,
    'Boston12': 35, 'Chicago1': 27, 'Chicago5': 60, 'Chicago6': 70,
    'Chicago7': 76, 'Chicago8': 76, 'Chicago9': 68, 'Chicago10': 56,
    'Chicago11': 45, 'Chicago12': 32, 'Philadelphia1': 35, 'Philadelphia5': 66,
    'Philadelphia6': 76, 'Philadelphia7': 81, 'Philadelphia8': 79,
    'Philadelphia9': 72, 'Philadelphia10': 60, 'Philadelphia11': 49,
    'Philadelphia12': 40}

Visualizing data with matplot. Matplot is soooooo elitteeeee. 

In [2]:
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 14
pd.set_option('display.max_columns', 99)

Reformatting the completeDataset/combined (train+test) data into a compatible format through numpy, merging, etc. completeDataset data > segmented. 

In [6]:
# Validation Groups
completeDataset = completeDataset.merge(validation_splits, on='City')
completeDataset['ValidationGroup'] = 1
completeDataset.loc[completeDataset.Latitude <= completeDataset.l1, 'ValidationGroup'] = 0
completeDataset.loc[completeDataset.Latitude > completeDataset.l2, 'ValidationGroup'] = 2
completeDataset.drop(['l1', 'l2'], axis=1, inplace=True)

adding headings, rounding the numbers to make them easier to process, figuring out where the missing values are, just general processing that's p self explanatory

In [8]:
completeDataset['Latitude3'] = completeDataset.Latitude.round(3)
completeDataset['Longitude3'] = completeDataset.Longitude.round(3)
completeDataset['EntryStreetMissing'] = 1 * completeDataset.EntryStreetName.isna()
completeDataset['ExitStreetMissing'] = 1 * completeDataset.ExitStreetName.isna()

completeDataset['CMWH'] = completeDataset.City + '_' \
               + completeDataset.Month.astype(str) + '_' \
               + completeDataset.Weekend.astype(str) + '_' \
               + completeDataset.Hour.astype(str)

completeDataset.EntryHeading = completeDataset.EntryHeading.replace(directionCodes)
completeDataset.ExitHeading = completeDataset.ExitHeading.replace(directionCodes)
completeDataset['DiffHeading'] = completeDataset['EntryHeading'] - completeDataset['ExitHeading']

In [ ]:
Combining city characteristics to use in model. self explanatory

In [9]:
completeDataset['city_month'] = completeDataset["City"] + completeDataset["Month"].astype(str)
completeDataset["Rainfall"] = completeDataset['city_month'].replace(monthly_rainfall)
completeDataset["Temperature"] = completeDataset['city_month'].replace(monthly_temperature)
completeDataset.drop('city_month', axis=1, inplace=True)

encoding the roads

In [10]:
def road_encode(x):
    for road in road_encoding.keys():
        if road in x:
            return road_encoding[road]
    return 0

completeDataset = completeDataset.fillna(dict(EntryStreetName='Unknown Something',
                        ExitStreetName='Unknown Something'))

completeDataset['EntryType'] = completeDataset['EntryStreetName'].apply(road_encode)
completeDataset['ExitType'] = completeDataset['ExitStreetName'].apply(road_encode)

cleaning up the completeDataset datasets by adding "idk" values to all the discrepancies in the dataset

In [ ]:
completeDataset = completeDataset.fillna(dict(EntryStreetName='Unknown Something',
                        ExitStreetName='Unknown Something'))

completeDataset['EntryType'] = completeDataset['EntryStreetName'].apply(road_encode)
completeDataset['ExitType'] = completeDataset['ExitStreetName'].apply(road_encode)

reformatting street signs into compatible format

In [11]:
completeDataset.EntryStreetName = completeDataset.City + ' ' + completeDataset.EntryStreetName
completeDataset.ExitStreetName = completeDataset.City + ' ' + completeDataset.ExitStreetName
completeDataset['Intersection'] = completeDataset.City + ' ' + completeDataset.IntersectionId.astype(str)

completeDataset['SameStreet'] = 1 * (completeDataset.EntryStreetName == completeDataset.ExitStreetName)

using a formula to figure out the distance between the given lat/longs and distribution from the complete dataset. helps determine their divergence from the distribution, which will be really useful in the model. 

In [12]:
# Geolocation
for col in ['Latitude', 'Longitude']:
    scaler = StandardScaler()
    completeDataset[col] = scaler.fit_transform(completeDataset[col].values.reshape(-1, 1))

# Distance from CityCenter
completeDataset = completeDataset.merge(
    completeDataset.groupby('City')[['Latitude', 'Longitude']].mean(),
    left_on='City', right_index=True, suffixes=['', 'Dist']
)
completeDataset.LatitudeDist = (5 * np.abs(completeDataset.Latitude - completeDataset.LatitudeDist)).round(3)
completeDataset.LongitudeDist = (5 * np.abs(completeDataset.Longitude - completeDataset.LongitudeDist)).round(3)
completeDataset['CenterDistL1'] = (5 * (completeDataset.LatitudeDist + completeDataset.LongitudeDist)).round(3)
completeDataset['CenterDistL2'] = (3 * np.sqrt(
    (completeDataset.LatitudeDist ** 2 + completeDataset.LongitudeDist ** 2))).round(3)

combining everything into the complete dataset

In [13]:
def add_frequency(df, column):
    cnt = df.groupby(column)[['RowId']].count()
    cnt.loc[cnt.RowId > 10, 'RowId'] = 10 * (
            cnt.loc[cnt.RowId > 10, 'RowId'] // 10)
    cnt.columns = [f'{column}Count']
    return df.merge(cnt, left_on=column, right_index=True)

column_structures = ['Longitude3', 'Latitude3', 'ExitStreetName', 'EntryStreetName', 'Intersection', 'Path']

for x in range(0,5): 
    completeDataset = add_frequency(completeDataset, column_structures[x])

# Frequency Encoding with unique intersections
def add_unique_intersections(df, column):
    cnt = df.groupby(column)[['Intersection']].nunique()
    cnt.loc[cnt.Intersection > 10, 'Intersection'] = 5 * (
            cnt.loc[cnt.Intersection > 10, 'Intersection'] // 5)
    cnt.columns = [f'{column}UniqueIntersections']
    return df.merge(cnt, left_on=column, right_index=True)

geo_column_structures = ['Longitude3', 'Latitude3', 'ExitStreetName', 'EntryStreetName']
for x in range(0,3): 
    completeDataset = add_unique_intersections(completeDataset, geo_column_structures[x])


Encoding the really relevant columns

In [14]:
columns_to_encode = ['City','EntryStreetName','ExitStreetName','Intersection', 'CMWH']

for c in columns_to_encode:
    encoder = LabelEncoder()
    completeDataset[c] = encoder.fit_transform(completeDataset[c])

exporting the completeDataset dataset csv out. compressed as gz bc upload cant handle the huge file size. 

In [15]:
completeDataset.to_csv('./data/features_v3.csv.gz', compression='gzip', index=False)

finalizing the stuff by concating into one pd file that is exported to col_stats.csv

In [16]:
trainDataset = completeDataset[completeDataset.Istrain == 1].copy()
test = completeDataset[completeDataset.Istrain == 0].copy()

column_stats = pd.concat([
    pd.DataFrame(completeDataset.count()).rename(columns={0: 'cnt'}),
    pd.DataFrame(trainDataset.count()).rename(columns={0: 'trainDataset_cnt'}),
    pd.DataFrame(test.count()).rename(columns={0: 'test_cnt'}),
    pd.DataFrame(completeDataset.nunique()).rename(columns={0: 'unique'}),
    pd.DataFrame(trainDataset.nunique()).rename(columns={0: 'trainDataset_unique'}),
    pd.DataFrame(test.nunique()).rename(columns={0: 'test_unique'}),
], sort=True, axis=1)


column_stats['seen_in_trainDataset%'] = (
            100 * column_stats.trainDataset_unique / column_stats.unique).round(1)
column_stats = column_stats.sort_values(by='unique')


column_stats.to_csv('data/col_stats.csv')

,cnt,train_cnt,test_cnt,unique,train_unique,test_unique,seen_in_train%
Weekend,2777744,856387,1921357,2,2,2,100.0
IsTrain,2777744,856387,1921357,2,1,1,50.0
ExitStreetMissing,2777744,856387,1921357,2,2,2,100.0
EntryStreetMissing,2777744,856387,1921357,2,2,2,100.0
SameStreet,2777744,856387,1921357,2,2,2,100.0
ValidationGroup,2777744,856387,1921357,3,2,3,66.7
City,2777744,856387,1921357,4,4,4,100.0
ExitType,2777744,856387,1921357,5,5,5,100.0
EntryType,2777744,856387,1921357,5,5,5,100.0
ExitHeading,2777744,856387,1921357,8,8,8,100.0
